# TP LDA

## Bibliothèques

In [7]:
import gensim
from gensim.models import Word2Vec
import sklearn
from  sklearn.manifold import TSNE
import numpy as np

# LDA 
import pandas as pd
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim.models import CoherenceModel
import nltk
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\rymkm\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\rymkm\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

## Import des données 

In [8]:
data = pd.read_csv('data_new_fake_theonion.csv')
data.head()

,Unnamed: 0,articles
0,0,"DAYTON, OH—While greeting the crowd at a campa..."
1,1,Presidential candidate Donald Trump recently r...
2,2,And the RNC is going to pay for it. It probabl...
3,3,"VANDALIA, OH—Drawing criticism for what many c..."
4,4,WASHINGTON—Her mind spinning as she poured ove...


## Nettoyage 

In [9]:
#Supprimer les lignes 
data = data.dropna(subset=['articles'])
#Supprimer les doublons
data = data.drop_duplicates(subset=['articles'])

**Garder la colonne qui nous interesse**

In [10]:
data = data['articles']
data.head()

0    DAYTON, OH—While greeting the crowd at a campa...
1    Presidential candidate Donald Trump recently r...
2    And the RNC is going to pay for it. It probabl...
3    VANDALIA, OH—Drawing criticism for what many c...
4    WASHINGTON—Her mind spinning as she poured ove...
Name: articles, dtype: object

## Tokenisation Stopwords Lemmatisation et Stemming 

**Charger la fonction**

In [11]:
stemmer = SnowballStemmer('english')

def lemmatize_stemming(text) :
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='n'))

def preprocess(text) :
    result = []
    for token in gensim.utils.simple_preprocess(text) :
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3 :
            result.append(lemmatize_stemming(token))
    return result

processed_docs = [preprocess(doc) for doc in data]

In [12]:
processed_docs

[['dayton',
  'greet',
  'crowd',
  'campaign',
  'ralli',
  'thursday',
  'presid',
  'donald',
  'trump',
  'seen',
  'kiss',
  'support',
  'burger',
  'accord',
  'sourc',
  'attend',
  'juici',
  'littl',
  'ask',
  'presidenti',
  'candid',
  'report',
  'lift',
  'fulli',
  'load',
  'flame',
  'broil',
  'beef',
  'hamburg',
  'hand',
  'press',
  'lip',
  'sesam',
  'seed',
  'emit',
  'audibl',
  'muah',
  'ounc',
  'wish',
  'vote',
  'ador',
  'littl',
  'tasti',
  'burger',
  'aren',
  'aren',
  'precious',
  'bite',
  'right',
  'remind',
  'sandwich',
  'home',
  'sourc',
  'confirm',
  'trump',
  'handler',
  'eventu',
  'interven',
  'releas',
  'burger',
  'grip',
  'ralli',
  'attende',
  'finish',
  'eat'],
 ['presidenti',
  'candid',
  'donald',
  'trump',
  'recent',
  'remark',
  'lose',
  'elect',
  'bloodbath',
  'countri',
  'onionexamin',
  'meant',
  'statement',
  'pretti',
  'straightforward',
  'honest',
  'stab',
  'stab',
  'stab',
  'sharp',
  'blood',

**Calcul de l'embeding moyen**

**Fixer un seuil**

## LDA 

### Transformation en bag of word

In [14]:
#tester les autres dictionnaires
dictionary = gensim.corpora.Dictionary(processed_docs)

dictionary.filter_extremes(no_below=15, no_above=0.1, keep_n=1000)

bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

dictionary

### Chargement du model 

In [16]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics = 5, id2word = dictionary, passes = 1000)

### Faire un save

In [17]:
#pour faire save
lda_model.save('lda_model')

### Separation en contextes

In [18]:
topics = []
for idx, topic in lda_model.print_topics(-1) :
    print("Topic: {} -> Words: {}".format(idx, topic))

topics.append(topic)

Topic: 0 -> Words: 0.028*"court" + 0.022*"justic" + 0.020*"mueller" + 0.018*"suprem" + 0.018*"investig" + 0.014*"eric" + 0.013*"special" + 0.012*"robert" + 0.010*"counsel" + 0.010*"russia"
Topic: 1 -> Words: 0.010*"congress" + 0.009*"repres" + 0.009*"speaker" + 0.008*"capitol" + 0.007*"obama" + 0.007*"mcconnel" + 0.007*"stand" + 0.007*"conserv" + 0.007*"pelosi" + 0.006*"speech"
Topic: 2 -> Words: 0.010*"room" + 0.009*"vice" + 0.009*"fuck" + 0.007*"penni" + 0.006*"desanti" + 0.006*"night" + 0.006*"chief" + 0.006*"harri" + 0.006*"shit" + 0.006*"away"
Topic: 3 -> Words: 0.010*"administr" + 0.008*"govern" + 0.008*"child" + 0.008*"woman" + 0.007*"order" + 0.006*"secretari" + 0.006*"provid" + 0.005*"critic" + 0.005*"execut" + 0.005*"allow"
Topic: 4 -> Words: 0.025*"poll" + 0.015*"race" + 0.014*"debat" + 0.014*"clinton" + 0.014*"ballot" + 0.010*"issu" + 0.010*"sander" + 0.009*"number" + 0.008*"critic" + 0.008*"elector"


### Coherence du modele

In [19]:
coherence_model_lda = CoherenceModel(model=lda_model,
texts=processed_docs, dictionary=dictionary)
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda)

Coherence Score:  0.33227453815980407


### Stockage du résultat

In [20]:
all_topic_model = []

for i in range(len(topics)):
    str = topics[i].split(' + ')
    topic_model = []
    for j in range(10):
        weight = str[j][0:5]
        word = str[j][7:len(str[j])-1]
        topic_model.append((weight, word))
    all_topic_model.append(topic_model)

In [21]:
df_topic_model = pd.DataFrame(all_topic_model)

df_topic_model.rename(index = {0: "Topic 1", 1: "Topic 2", 2:"Topic 3", 3: "Topic 4", 4: "Topic 5", 5: "Topic 6", 6: "Topic 7", 7:"Topic 8", 8: "Topic 9", 9: "Topic 10"})

,0,1,2,3,4,5,6,7,8,9
Topic 1,"(0.025, poll)","(0.015, race)","(0.014, debat)","(0.014, clinton)","(0.014, ballot)","(0.010, issu)","(0.010, sander)","(0.009, number)","(0.008, critic)","(0.008, elector)"


### Representation 2D

In [22]:
import pyLDAvis.gensim_models

pyLDAvis.enable_notebook()
pyLDAvis.gensim_models.prepare(lda_model, bow_corpus,dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.082997  0.104405       1        1  26.376786
2     -0.101254 -0.159511       2        1  25.882253
1      0.075494  0.015482       3        1  19.903258
4      0.122015 -0.062676       4        1  17.168720
0     -0.179252  0.102301       5        1  10.668983, topic_info=        Term        Freq       Total Category  logprob  loglift
258     poll  314.000000  314.000000  Default  30.0000  30.0000
318    court  222.000000  222.000000  Default  29.0000  29.0000
557   justic  176.000000  176.000000  Default  28.0000  28.0000
999  mueller  156.000000  156.000000  Default  27.0000  27.0000
716   suprem  146.000000  146.000000  Default  26.0000  26.0000
..       ...         ...         ...      ...      ...      ...
117    legal   53.226690  135.480472   Topic5  -5.0014   1.3036
333     hear   56.160793  180.813086   Topic5  -4.9478   1.0686
625    email   34.308727   82.083627   Topic5  -5.4406   1.3655
498  general   35.940350  111.782924   Topic5  -5.3941   1.1031
371     read   35.191026  139.319773   Topic5  -5.4152   0.8618

[272 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
709       1  0.876298    abus
709       3  0.046121    abus
709       5  0.069181    abus
764       1  0.958584  access
764       5  0.042290  access
...     ...       ...     ...
385       5  0.031136   world
368       1  0.137795    york
368       2  0.291489    york
368       4  0.535280    york
368       5  0.037099    york

[653 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 3, 2, 5, 1])